### Homework 3
Please explain clearly and include your entire computational work when needed. Should you include any code, please make sure to provide additional comments to explain your solution. 

---

#### Q1 (30 points) Hyper-parameter Tuning: Provide your coding for this exercise. Your code should run without error and the output should be correct based on your assumptions. 

- Create a dummy dataset with 3 input features (X) and a numeric output variable (y). Your daatset should have 300 data points. The train:test split is 75:25. 
- Standardize the input features. 
- Construct a 3-layer neural network with 2 neurons in the hidden layer and one neuron in the output layer. Use Relu as the activation function in the hidden layer. Use Sigmoid as the activation function in the output layer. 
- At this step, you are to use grid search to perform hyper-parameter tuning based on regularization and regularization parameter, momentum term, and batch size. For each paramter, try 2 different values. Refer to "lec11-grid-search.ipynb" as a reference. You are not allowed to copy-paste the code to solve this assignment. Just use it to learn what to do. Use SGD as your optimization technique. 
- In your code, report which subset of hyper-parameters give the best model. Then using the best hyper-parameters that you reported, visualize the MSE loss on train and test data over multiple epochs (the number of epochs depends on model convergence). 

Note: Hyperparameter tuning takes a lot of time to execute. Make sure that you choose the appropriate number of each hyperparameter (preferably 3 of each), and that you allocate enough time to execute your code.

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.neural_network import MLPRegressor
import matplotlib.pyplot as plt

np.random.seed(42)

# dummy dataset
size = 300
data = np.random.randn(size, 3) 
target = 2*data[:,0] + 0.5*data[:,1] - data[:,2] + np.random.randn(size)*0.1 

# train:test split data
d_train, d_test, t_train, t_test = train_test_split(data, target, test_size=0.25, random_state=42)

# standardize input features
scaler = StandardScaler()
d_train_norm = scaler.fit_transform(d_train)
d_test_norm = scaler.transform(d_test)

# neural network model
nn = MLPRegressor(
    hidden_layer_sizes=(2,),
    activation='relu',
    solver='sgd',
    random_state=42,
    max_iter=1,
    warm_start=True         
)

# parameter grid for grid search
params = {
    'alpha': [0.01, 0.1], 
    'momentum': [0.8, 0.9],
    'batch_size': [16, 32],
    'learning_rate_init': [0.01, 0.1]
}

# grid search
grid = GridSearchCV(
    estimator=nn,
    param_grid=params,
    cv=3,
    n_jobs=-1,
    verbose=2
)
grid_out = grid.fit(d_train_norm, t_train)

# best model 
best = grid_out.best_params_
model = MLPRegressor(
    hidden_layer_sizes=(2,),
    activation='relu',
    solver='sgd',
    alpha=best['alpha'],
    batch_size=best['batch_size'],
    learning_rate_init=best['learning_rate_init'],
    momentum=best['momentum'],
    random_state=42,
    max_iter=1,
    warm_start=True
)

# train and track losses
tr_losses = []
val_losses = []
epochs = 100

for _ in range(epochs):
    model.partial_fit(d_train_norm, t_train)
    
    tr_pred = model.predict(d_train_norm)
    tr_loss = np.mean((tr_pred - t_train) ** 2)
    tr_losses.append(tr_loss)
    
    val_pred = model.predict(d_test_norm)
    val_loss = np.mean((val_pred - t_test) ** 2)
    val_losses.append(val_loss)

final_pred = model.predict(d_test_norm)
final_loss = np.mean((final_pred - t_test) ** 2)

plt.figure(figsize=(10, 6))
plt.plot(tr_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.title('Model Loss with Best Hyperparameters')
plt.xlabel('Epoch')
plt.ylabel('Mean Squared Error')
plt.legend()
plt.grid(True)
plt.show()

print(f"\nBest when:")
print(f"Regularization (alpha) is {best['alpha']}")
print(f"Momentum is {best['momentum']}")
print(f"Batch size is {best['batch_size']}")
print(f"Learning rate is {best['learning_rate_init']}")
print(f"Final test loss (MSE): {final_loss:.4f}")

#### Q2- (30 points) Given the information below, use Naïve Bayes to determine whether File 6 belongs to class label "Spam" or "Ham" given its features. Answering following questions clearly and accurately. Provide your computational work. 

| File | Category | Content
| ---- | -------- | -------
| 1    | Spam     | Price, Ticket, Free
| 2    | Ham      | Tax, Price, Puppy
| 3    | Ham      | Free, Puppy, Ticket, Price
| 4    | Spam     | Ticket, Tax, Puppy
| 5    | Ham      | Price, Tax, Puppy
| 6    | ?        | Price, Tax, Free


#### What are the prior probabilities P(Spam), P(Ham)? Compute the required conditional probabilitues corresponding to the parameters of the model by filling the table below. 

| Word   | P(word/Spam) | P(word/Ham)
| ------ | ------------ | --------------
| Price  | 1/6          | 3/10
| Ticket | 2/6          | 1/10
| Free   | 1/6          | 1/10
| Tax    | 1/6          | 2/10
| Puppy  | 1/6          | 3/10


#### What is the probability of File 6 being a Spam? What is the probability of File 6 being a Ham? What can you conclude about the category of File 6? Justify your answer. 

P(Spam|Price, Tax, Free) ⟶ P(Price|Spam) × P(Tax|Spam) × P(Free|Spam) × P(Spam) = 1/6 × 1/6 × 1/6 × 0.4 = 0.00185
P(Ham|Price, Tax, Free) ⟶ P(Price|Ham) × P(Tax|Ham) × P(Free|Ham) × P(Ham) = 3/10 × 2/10 × 1/10 × 0.6 = 0.0036

Because 0.0036 is greater than 0.00185, file 6 is more likely to be ham.

---

#### Q3 - (6 points) A patient with a positive test wants to know how probable it is to have flu. Find the probability given the information below and provide your computational work. 
- Prior probability: P(not flu) = 0.15
- Likelihood: P(positive│flu) = 0.85
- Evidence: P(negative) = 0.25

P(flu|positive) = (P(positive|flu) × P(flu))/P(positive) = (P(positive|flu) × (1 - P(not flu)))/(1 - P(positive)) = (0.85 × 0.85)/(.75) = 0.963
P(flu|positive) = 96.3%

---

#### Q4 - (8 points) Imagine you are tasked with developing a binary classification model to classify emails as either "spam" or "not spam" based on their content. The observations in the dataset cover several aspects listed in the following table: 


![Generated Dataset](./tbl3.jpg)

You have two different approaches in mind: Naive Bayes classification and Bayesian networks. Answer the following question in the context of this email classification (as “spam” or “not spam”): 
Discuss the assumptions in Naïve Bayes and Bayesian Network algorithms, as well as advantages and disadvantages of using Naïve Bayes and Bayesian Network classification. Your answer should address the following points: 
-	What aspects of the problem make Naïve Bayes classification suitable or unsuitable? 
-	Similarly, analyze the suitability of using Bayesian networks for the email classification problem and indicate what unique characteristics of Bayesian networks make them a potentially good or bad choice for this task? 
-	Discuss these methods in terms of their computational efficiency and classification accuracy, considering the features listed in the above table. 


| Approach          | Assumptions  | Advantages     | Disadvantages
| ----------------- | ------------ | -------------- | --------------
| Naive Bayes       | Word order doesn't matter, each feature contributes independently to spam probability | Simple + fast to implement, works well with high-dimensional data (lots of features), effective with smaall training datasets, computationally efficient, good for text classsification | Independence assumption is sometimes unrealistic, can miss important relationships between features and be oversensitive to irrelevant features
| Bayesian Networks | Relationships between features represented as directed acyclic graph, conditional dependencies explicitly modeled, trie casual relationships are reflected in network structure | Can model complex dependencies between features, more accurate when features are related, can incorporate domain knowledge in network structure, better handling of missing data, can represent causal relationships | More complicated to implement, computationally expensive, requires larger training datasets, network structure can be difficult to determine, learning complexity goes up with network size |

For email classification, Naive Bayes is more suitable because the words and metadata of an email can be treated independently. Text data is often high dimensional, and just a simple word frequency analysis is fine for spam detection. 
Bayesian Networks are less suitable for email classification because the network formed would be too complex given all the features in the table and would be computationally expensive. 

#### Q5 - (6 points) Given the following probability table, build a Naïve Bayes classifier using the Discriminant function. Can this classifier accurately predict if a patient is healthy/not-healthy based on the Test results? (e.g., can a positive test result indicate with high confidence that a patient has flu? How about a negative test result?) Show your mathematical work with proper explanation. Please note that P(Healthy) = 0.25. 
![Generated Dataset](./tbl5.jpg)


G(Positive) = log[P(Positive|Has-Flu)×P(Has-Flu)] - log[P(Positive|Healthy)×P(Healthy)]\
= log(0.8 × 0.75) - log(0.1 × 0.25)\
= log(0.6) - log(0.025)\
= 1.3802 > 0 --> classify as Has-Flu, large positive + strong confidence

G(Negative) = log[P(Negative|Has-Flu)×P(Has-Flu)] - log[P(Negative|Healthy)×P(Healthy)]\
= log(0.2 × 0.75) - log(0.9 × 0.25)\
= log(0.15) - log(0.225)\
= -0.1761 < 0 --> classify as Healthy, moderate negative + medium confidence

#### Q6 - (Optional Question- 8 extra points based on the completeness of the answer) Based on the optional document "Lec12-MultivariateDistributions.pdf", describe how conditional probabilities are computed in training a Bayesian Network model when the random variables are continuous. Provide an example.  

In Bayesian Networks with continuous variables, conditional probabilities are found using probability density functions (PDFs). To find a conditional probability, we divide the joint PDF by the marginal PDF and integrate over the relevant variables.
We'll use this the example of tracking daily market returns (X₁) and stock prices (X₂). If both daily market returns and stockkk prices follow a normal distribution, in order to find the probability of a stock price given market returns, P(X₂|X₁), we would:

1. Get the joint PDF from historical data
2. Calculate the marginal PDF by integrating over X₂
3. Divide the joint PDF by the marginal PDF

We would be able to observe a continuous relationship between market returns and stock prices, rather than discrete probability values.